In [1]:
from dotenv import load_dotenv
from huggingface_hub import login
import os

load_dotenv()
hftoken = os.getenv("HUGGINGFACE_TOKEN")
login(hftoken)

In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
import random
from litellm import completion, batch_completion
import os
import litellm
import time
from tqdm import tqdm

# OpenAI API key 선언
#os.environ["OPENAI_API_KEY"] = "sk-xxx..."

In [27]:
df

,question,response,query
0,포트폴리오 구성에서 동등 가중치(equal-weighted)와 시가총액 가중치(ca...,Error: Rate limit exceeded or other issue,What are the portfolio construction difference...
1,다음 함수를 고려하십시오. \n\n최적의 거래에 대한 기대 로그 수익률을 측정하고자...,"주어진 문제를 해결하기 위해, 매수 및 매도 포지션의 기대 로그 수익률을 더욱 매끄...",See the function here.\nIt's my intent to meas...
2,주식이 다음과 같은 확률적 미분 방정식을 따른다고 가정하자.\n$$dS=\frac{...,"주어진 확률적 미분 방정식을 기반으로 옵션의 가치 \( V(t, S_t) \)의 이...",Suppose a stock follows the stochastic differe...
3,실제로 불완전 모델에 대해 가격 구간을 계산하는 경우가 있을까요? 만약 그렇다면 그...,불완전 모델에 대한 가격 구간을 계산하는 경우는 종종 있습니다. 이러한 경우는 주로...,If a market model is complete then every deriv...
4,Error: Rate limit exceeded or other issue,It seems you're encountering a technical issue...,Closed. This question is off-topic. It is not ...
...,...,...,...
195,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue,I am new on this forum and i have just begun m...
196,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue,How do we determine which curve to use for pri...
197,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue,I need help understanding why there are many d...
198,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue,Below is an attached screenshot of a scatter p...


1. query 컬럼으로 질문 생성 및 생성된 질문에 대한 답변 생성

In [26]:

def random_wait(min_seconds=0.1, max_seconds=0.5):
    wait_time = random.uniform(min_seconds, max_seconds)
    print(f"Waiting for {wait_time:.2f} seconds...")
    time.sleep(wait_time)

# 데이터셋 로드
ds = load_dataset('amphora/rewrite-se-quant')['train']

# 전체 데이터셋을 미리 데이터프레임으로 변환
df_ds = pd.DataFrame(ds)

# 데이터프레임 초기화
df = pd.DataFrame(columns=['query', 'question', 'response'])


batch_size = 200
total_length = len(ds['query'])

# 반복문 안에서 현재 배치의 데이터프레임 생성 및 기존 데이터프레임에 추가
for i in tqdm(range(3600, total_length, batch_size), desc="Processing Batches", unit="batch"):
    batch_end = min(i + batch_size, total_length)  # 끝 인덱스가 전체 길이를 넘지 않도록 설정

    # 질문 생성용 prompt 포맷팅
    batch_qrys = []
    for t in df_ds['query'][i:batch_end]:
        messages = [
            {"content": "Your job is creating quantitative finance questions in fluent Korean. You will be given a English QF question collected from the web. Restructure it to a test-like question, in formal Korean language. Return the question only.", "role": "system"},
            {"content": t, "role": "user"}
        ]
        batch_qrys.append(messages)

    # 질문 생성
    responses = batch_completion(
        model="gpt-4o-mini-2024-07-18",
        messages=batch_qrys,
        # max_tokens=60
    )

    question_resps = []
    for response in responses:
        if isinstance(response, litellm.RateLimitError):
            print(f"오류 발생: {str(response)}")
            question_resps.append("Error: Rate limit exceeded or other issue")
        elif hasattr(response, 'choices') and len(response.choices) > 0:
            question_resps.append(response.choices[0].message.content)
        else:
            question_resps.append("Error: Unexpected response format")

    # 질문 생성과 답변 생성 사이에 10초 간격 추가
    print(i, '1분 대기')
    time.sleep(60)
    print('대기 종료')

    # 답변 생성용 prompt 포맷팅
    batch_qrys = []
    for t in question_resps:
        messages = [
            {"content": "You are a skilled financial expert in Korea. Make a response for the question. DO NOT introduce yourself.", "role": "system"},
            {"content": t, "role": "user"}
        ]
        batch_qrys.append(messages)

    # 답변 생성
    responses = batch_completion(
        model="gpt-4o-mini-2024-07-18",
        messages=batch_qrys,
        # max_tokens=80
    )

    response_resps = []
    for resp in responses:  # 'i' 대신 'resp'로 변수명 변경
        if isinstance(resp, Exception):
            print(f"오류 발생: {str(resp)}")
            response_resps.append("Error: Rate limit exceeded or other issue")
        elif hasattr(resp, 'choices') and len(resp.choices) > 0:
            response_resps.append(resp.choices[0].message.content)
        else:
            response_resps.append("Error: Unexpected response format")
        random_wait()


    print(i, '5초 대기')
    time.sleep(5)
    print('대기 종료')

    query_list = df_ds['query'].iloc[i:batch_end].tolist()

    # 현재 배치의 데이터프레임 생성 및 기존 데이터프레임에 추가
    batch_df = pd.DataFrame({
        'query': query_list,
        'question': question_resps,
        'response': response_resps
    })
    df = pd.concat([df, batch_df], ignore_index=True)

    print('total prompt tokens:', total_prompt_tokens_for_q + total_prompt_tokens_for_a)
    print('prompt token costs:', round((total_prompt_tokens_for_q + total_prompt_tokens_for_a) / 1000000 * 0.150, 6))
    print('total completion tokens:', total_completion_tokens_for_q + total_completion_tokens_for_a)
    print('completion token costs:', round((total_completion_tokens_for_q + total_completion_tokens_for_a) / 1000000 * 0.600, 6))
    print('total completion tokens_q:', total_completion_tokens_for_q)
    print('total completion tokens_a:', total_completion_tokens_for_a)

    # 원본 데이터셋의 query 길이
    original_length = len(ds["query"])

    # 생성된 질문과 답변의 길이 확인
    question_length = len(question_resps)
    response_length = len(response_resps)

    print(f"Original length: {original_length}")
    print(f"Generated question length: {question_length}")
    print(f"Generated response length: {response_length}")

    # 길이가 다른지 확인
    if question_length < original_length:
        print(f"Missing questions: {original_length - question_length}")

    if response_length < original_length:
        print(f"Missing responses: {original_length - response_length}")

    # 다음 배치를 처리하기 전 1분 간격 대기
    print(i, '1분 대기')
    time.sleep(60)
    print('대기 종료')

# 최종 결과 확인
df.head()

Processing Batches:   0%|          | 0/92 [00:00<?, ?batch/s]/home/elicer/.local/lib/python3.10/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `CompletionTokensDetails` but got `dict` with value `{'audio_tokens': None, 'reasoning_tokens': 0}` - serialized value may not be as expected
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 0}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.I

Processing Batches:   0%|          | 0/92 [03:23<?, ?batch/s]

대기 종료


NameError: name 'total_prompt_tokens_for_q' is not defined

2. Qwen으로 질문 및 답변 생성 (너무 오래걸림)

In [6]:
import pandas as pd
import time
from tqdm import tqdm
from datasets import load_dataset
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm

def qwen_completion(model_name, inputs, max_new_tokens=60):
    # Load the tokenizer and model for Qwen
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

    results = []
    for prompt in tqdm(inputs):
        # Tokenize the prompt
        inputs = tokenizer(prompt, return_tensors="pt")

        # Generate text using max_new_tokens instead of max_length
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)

        # Decode the generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        results.append({'generated_text': generated_text})
    return results

# 데이터셋 로드
ds = load_dataset('amphora/rewrite-se-quant')['train']

# 전체 데이터셋을 미리 데이터프레임으로 변환
df_ds = pd.DataFrame(ds)

# 데이터프레임 초기화 ('query' 컬럼 제거)
df = pd.DataFrame(columns=['question', 'response'])

# 전체 데이터를 대상으로 질문 생성용 prompt 포맷팅
batch_qrys = []
for t in tqdm(df_ds['query'], desc="Processing Questions"):
    # Qwen 모델의 입력 포맷에 맞게 프롬프트 생성
    prompt = f"문제: {t}\n질문을 한국어로 만들어주세요."
    batch_qrys.append(prompt)

# 질문 생성 - Qwen 모델 사용
question_resps = qwen_completion(
    model_name="Qwen/Qwen2-7B-Instruct",  # Qwen 모델 이름
    inputs=batch_qrys,
    max_new_tokens=60
)

# Qwen 모델에서 받은 응답 처리
question_resps = [resp['generated_text'] if 'generated_text' in resp else "Error: Unexpected response format" for resp in question_resps]

# 전체 데이터를 대상으로 답변 생성용 prompt 포맷팅
batch_qrys = []
for t in tqdm(question_resps, desc="Processing Responses"):
    # Qwen 모델의 입력 포맷에 맞게 프롬프트 생성
    prompt = f"질문: {t}\n답변을 만들어주세요."
    batch_qrys.append(prompt)

# 답변 생성 - Qwen 모델 사용
response_resps = qwen_completion(
    model_name="Qwen/Qwen2-7B-Instruct",  # Qwen 모델 이름
    inputs=batch_qrys,
    max_new_tokens=80
)

# Qwen 모델에서 받은 응답 처리
response_resps = [resp['generated_text'] if 'generated_text' in resp else "Error: Unexpected response format" for resp in response_resps]

# 최종 데이터프레임 생성
df = pd.DataFrame({
    'question': question_resps,
    'response': response_resps
})

# 최종 결과 확인
df.head()


Processing Questions: 100%|██████████| 21950/21950 [00:00<00:00, 805171.96it/s]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 3/21950 [03:27<421:24:41, 69.12s/it]


KeyboardInterrupt: 

In [ ]:
print(f"중복된 행의 개수: {df.duplicated().sum()}")

In [ ]:
# 질문 생성 결과 길이 맞추기
if len(question_resps) < len(ds["query"]):
    question_resps.extend(["Error: Missing question"] * (len(ds["query"]) - len(question_resps)))

# 답변 생성 결과 길이 맞추기
if len(response_resps) < len(ds["query"]):
    response_resps.extend(["Error: Missing response"] * (len(ds["query"]) - len(response_resps)))


In [ ]:
# Excel 파일 저장
# df.to_excel("output_path/result.xlsx")

# HuggingFace Hub 업로드 - token에 개인 HuggingFace 토큰을 입력해주시면 됩니다.
result_df = Dataset.from_pandas(df)
result_df.push_to_hub("LDC-ai/dataset", token=hftoken)